In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import py3Dmol

from proteome import protein
from proteome.models.proteinmpnn import config
from proteome.models.proteinmpnn.modeling import ProteinMPNNForSequenceDesign, PROTEINMPNN_MODEL_URLS
from proteome.models.omegafold.modeling import OmegaFoldForFolding

In [ ]:
designer = ProteinMPNNForSequenceDesign("vanilla_model-20", random_seed=0)
folder = OmegaFoldForFolding()

In [ ]:
with open("5L33.pdb", mode="r") as f:
    gt_pdb = f.read()

In [ ]:
target_protein = protein.from_pdb_string(
    gt_pdb, ca_only=designer.ca_only, backbone_only=(not designer.ca_only)
)

In [ ]:
sequence, score = designer.design_sequence(target_protein)
print(f"Sequence: {sequence} with global_score {score}")

In [ ]:
predicted_protein, confidence = folder.fold(sequence)
result_pdb = protein.to_pdb(predicted_protein)

In [ ]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(result_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()